In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tenfem
from scipy.integrate import quad

In [2]:
tri_mesh = tenfem.mesh.examples.square(2, 2)
tri_element = tenfem.reference_elements.TriangleElement(degree=1)
# tri_element.get_quadrature_nodes(tri_mesh)

In [3]:
n_nodes = 15
a, b = -1.3, 0.5
nodes = np.linspace(a, b, n_nodes)[:, np.newaxis]
elements = np.column_stack((np.arange(0, n_nodes-1), np.arange(1, n_nodes)))
boundary_elements = [[0, ], [n_nodes-1, ]]

mesh = tenfem.mesh.IntervalMesh(nodes, elements, boundary_elements)

# Some words of caution

We have taken our reference element to be $[0, 1]$ to easily define the shape functions,
but the Gauss quadrature nodes are on the interval therefore over each element
we must adjust be the scaling factor 

$$
\frac{(b_i - a_i)}{2}
$$

this is automatically taken care of in the `quadrature` method.

In [11]:
#ref_element = tenfem.reference_elements.IntervalElement(# .get_quadrature_nodes(mesh)
ref_element = tenfem.reference_elements.IntervalElement(degree=1, dtype=tf.float64)
quad_nodes, quad_weights = ref_element.get_quadrature_nodes_and_weights(mesh)

In [18]:
def f(x):
    return x[..., 0]

def quadrature(self, f, mesh):
    """ Perform quadrature of a function over the mesh. 
    
    Args:
        f: A scalar callable, when evaluated at a mesh node of shape 
          `[..., spatial_dim]` it should return a `Tensor` of shape
          `[..., ]` with the same `dtype` as `mesh.
    
    Returns
        integral: A quadrature approximation to the integral of
          f over the mesh.
    """
    quad_nodes, quad_weights = self.get_quadrature_nodes_and_weights(mesh)
    f_at_nodes = f(quad_nodes)
    volumes = self.get_element_volumes(mesh)
    return 0.5 * tf.reduce_sum(f_at_nodes 
                               * quad_weights
                               * volumes[..., tf.newaxis], axis=[-1, -2])
    

f_at_nodes = f(quad_nodes)
quad_integral = tf.reduce_sum(f_at_nodes * quad_weights[...], axis=-1)

vols = np.diff(nodes[:, 0])[:, tf.newaxis]

quad_integral = tf.reduce_sum(quad_integral * vols) * .5

I, err = quad(lambda x: x, a, b)
print(I, quad_integral)
print(quadrature(ref_element, f, mesh))

-0.7200000000000002 tf.Tensor(-10.079999999999998, shape=(), dtype=float64)
tf.Tensor(-0.7200000000000001, shape=(), dtype=float64)


In [ ]:
rr = np.linspace(0., 1., 4)

In [ ]:
xx = np.linspace(0., 1.)
s, ds = ref_element.shape_function(xx)

fig, ax = plt.subplots()
ax.plot(xx, s[..., 0], label='$S_1$')
ax.plot(xx, s[..., 1], label='$S_2$')
ax.legend()
plt.show()